In [1]:
import pandas as pd
import os
import re
import sys

# Add the project root to the Python path
# This allows us to import the `config` module
project_root = os.path.abspath(os.path.join(os.path.dirname('__file__'), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

import config

In [2]:
def clean_text_basic(text):
    """
    Performs basic text cleaning:
    - Converts to string and handles NaN
    - Lowercases text
    - Replaces multiple spaces with a single space
    - Removes special characters, keeping alphanumeric, basic punctuation, and Amharic characters
    - Strips leading/trailing whitespace
    """
    if pd.isna(text):
        return ""
    text = str(text)
    text = text.lower()  # Lowercase
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with single
    # Keep alphanumeric, whitespace, and basic punctuation useful for NLP.
    # Added Amharic unicode range U+1200 to U+137F.
    text = re.sub(r'[^\w\s\.\?\!\'\u1200-\u137F]', '', text) 
    text = text.strip()
    return text

def preprocess_reviews(df_raw):
    """
    Orchestrates the entire preprocessing pipeline for the raw reviews DataFrame.
    """
    print(f"Initial number of reviews: {len(df_raw)}")
    df = df_raw.copy()

    # Ensure essential columns are present
    essential_cols = ['review_id', 'review_text', 'rating', 'date', 'bank_name', 'source']
    for col in essential_cols:
        if col not in df.columns:
            raise ValueError(f"Essential column '{col}' not found in raw data.")

    # Handle missing essential data
    df.dropna(subset=['review_text', 'rating', 'review_id'], inplace=True)
    print(f"Reviews after dropping NA in text/rating/review_id: {len(df)}")

    # Remove duplicates based on 'review_id'
    df.drop_duplicates(subset=['review_id'], keep='first', inplace=True)
    print(f"Reviews after dropping duplicates by review_id: {len(df)}")

    # Normalize dates (to YYYY-MM-DD)
    try:
        df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')
    except Exception as e:
        print(f"Warning: Could not parse all dates. Error: {e}. Coercing errors.")
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        df.dropna(subset=['date'], inplace=True)
        df['date'] = df['date'].dt.strftime('%Y-%m-%d')

    # Clean review text
    df['cleaned_review_text'] = df['review_text'].apply(clean_text_basic)

    # Filter out reviews with very short text after cleaning (e.g., less than 3 words)
    df = df[df['cleaned_review_text'].apply(lambda x: len(str(x).split()) > 2)]
    print(f"Reviews after filtering short cleaned texts: {len(df)}")

    # Select final columns
    final_columns = [
        'review_id', 'cleaned_review_text', 'rating', 'date', 'bank_name', 'source',
        'user_name', 'thumbs_up_count', 'app_version', 'review_text' # Keep original for reference
    ]
    # Only include columns that actually exist in the df
    final_columns = [col for col in final_columns if col in df.columns]
    df_cleaned = df[final_columns]
    
    return df_cleaned

In [3]:
# Define file paths using the config module
raw_file_path = os.path.join(config.RAW_DATA_DIR, "all_banks_reviews_raw.csv")
processed_file_path = os.path.join(config.PROCESSED_DATA_DIR, "all_banks_reviews_cleaned.csv")

# Check if raw data exists
if not os.path.exists(raw_file_path):
    print(f"Raw data file not found: {raw_file_path}. Please run scraper first.")
else:
    print(f"Found raw data at: {raw_file_path}")

# Create the processed data directory if it doesn't exist
if not os.path.exists(config.PROCESSED_DATA_DIR):
    os.makedirs(config.PROCESSED_DATA_DIR)
    print(f"Created directory: {config.PROCESSED_DATA_DIR}")

Found raw data at: data/raw/all_banks_reviews_raw.csv


In [4]:
# Load the raw data
df_raw = pd.read_csv(raw_file_path)

# Display basic information
print("Shape of the raw dataframe:", df_raw.shape)
print("\nFirst 5 rows of raw data:")
display(df_raw.head())

# Check initial missing data percentages
print("\nMissing data percentages before preprocessing:")
print((df_raw.isnull().sum() / len(df_raw)) * 100)

Shape of the raw dataframe: (9098, 9)

First 5 rows of raw data:


,review_id,review_text,rating,date,bank_name,source,user_name,thumbs_up_count,app_version
0,54f070b9-895f-40e8-be18-acb16f8af7fa,Best Mobile Banking app ever,5,2025-06-04 16:50:52,Commercial Bank of Ethiopia,Google Play,A Google user,0,5.0.4
1,aa72fd94-aca8-48b2-bae2-29ad7633c7fb,good,5,2025-06-04 16:00:15,Commercial Bank of Ethiopia,Google Play,A Google user,0,5.1.0
2,9af95d2a-e42c-45ff-a4c9-5c38765df4a4,it was good app but it have some issues like i...,2,2025-06-04 15:56:02,Commercial Bank of Ethiopia,Google Play,A Google user,0,5.1.0
3,067d3e52-3586-4e02-b088-0411f25d34fb,dedeb,5,2025-06-04 15:04:02,Commercial Bank of Ethiopia,Google Play,A Google user,0,NaN
4,88a64315-9de3-4a35-ac27-5ff37c2833b6,good,5,2025-06-04 14:59:40,Commercial Bank of Ethiopia,Google Play,A Google user,0,5.1.0



Missing data percentages before preprocessing:
review_id           0.000000
review_text         0.076940
rating              0.000000
date                0.000000
bank_name           0.000000
source              0.000000
user_name           0.000000
thumbs_up_count     0.000000
app_version        19.158057
dtype: float64


In [5]:
df_cleaned = preprocess_reviews(df_raw)

Initial number of reviews: 9098
Reviews after dropping NA in text/rating/review_id: 9091
Reviews after dropping duplicates by review_id: 9091
Reviews after filtering short cleaned texts: 5413


In [6]:
print(f"\nTotal cleaned reviews: {len(df_cleaned)}")
print("\nFirst 5 rows of cleaned data:")
display(df_cleaned.head())

# Calculate missing data percentages after cleaning
missing_percentage_cleaned = (df_cleaned.isnull().sum() / len(df_cleaned)) * 100
print("\nMissing data percentages after cleaning:")
print(missing_percentage_cleaned)

# KPI Check: We want less than 5% missing data in critical columns.
print("\n--- Data Quality KPI Check ---")
critical_cols_check = ['cleaned_review_text', 'rating', 'date', 'bank_name']
if missing_percentage_cleaned[critical_cols_check].max() < 5:
    print("✅ KPI Met: Less than 5% missing data in critical columns.")
else:
    print("⚠️ Warning: KPI NOT Met. More than 5% missing data in one or more critical columns.")
    print(missing_percentage_cleaned[critical_cols_check])


Total cleaned reviews: 5413

First 5 rows of cleaned data:


,review_id,cleaned_review_text,rating,date,bank_name,source,user_name,thumbs_up_count,app_version,review_text
0,54f070b9-895f-40e8-be18-acb16f8af7fa,best mobile banking app ever,5,2025-06-04,Commercial Bank of Ethiopia,Google Play,A Google user,0,5.0.4,Best Mobile Banking app ever
2,9af95d2a-e42c-45ff-a4c9-5c38765df4a4,it was good app but it have some issues like i...,2,2025-06-04,Commercial Bank of Ethiopia,Google Play,A Google user,0,5.1.0,it was good app but it have some issues like i...
8,fce00cda-d71a-486e-a4c2-7479ab7793bc,best app of finance,5,2025-06-04,Commercial Bank of Ethiopia,Google Play,A Google user,0,NaN,best app of finance
10,2470d199-834a-4134-a0f9-8c684ba75491,engida kebede fetera,5,2025-06-03,Commercial Bank of Ethiopia,Google Play,A Google user,0,5.1.0,Engida Kebede Fetera
12,428e5117-387f-4c9d-b095-3230e3f83a8a,it is not safety,1,2025-06-03,Commercial Bank of Ethiopia,Google Play,A Google user,0,5.1.0,it is not safety



Missing data percentages after cleaning:
review_id               0.000000
cleaned_review_text     0.000000
rating                  0.000000
date                    0.000000
bank_name               0.000000
source                  0.000000
user_name               0.000000
thumbs_up_count         0.000000
app_version            16.645114
review_text             0.000000
dtype: float64

--- Data Quality KPI Check ---
✅ KPI Met: Less than 5% missing data in critical columns.


In [7]:
print("\nReview counts per bank in the cleaned data:")
print(df_cleaned['bank_name'].value_counts())

# Save the cleaned dataframe
df_cleaned.to_csv(processed_file_path, index=False, encoding='utf-8')

print(f"\n✅ Cleaned data successfully saved to: {processed_file_path}")


Review counts per bank in the cleaned data:
bank_name
Commercial Bank of Ethiopia    4353
Bank of Abyssinia               657
Dashen Bank                     328
Enat Bank                        75
Name: count, dtype: int64

✅ Cleaned data successfully saved to: data/processed/all_banks_reviews_cleaned.csv
